# 数据处理、合并与分组  

🎈本节依然关注的是我们处理数据时最常用到的一些操作，比如：  
1. 缺失值的处理  
2. 合并数据集  
3. 根据某一种条件进行分组，并进行描述统计  

我们使用的依然是pandas120这个数据集，它包含了被试的学历和薪资水平等信息

In [1]:
#导包
import pandas as pd
import numpy as np

In [2]:
# 读取pandas120数据文件
df = pd.read_excel('/home/mw/input/pandas1206855/pandas120.xlsx')
df.head()

,createTime,education,salary
0,2020-03-16 11:30:18,本科,20k-35k
1,2020-03-16 10:58:48,本科,20k-40k
2,2020-03-16 10:46:39,不限,20k-35k
3,2020-03-16 10:45:44,本科,13k-20k
4,2020-03-16 10:20:41,本科,10k-20k


In [3]:
# 在数据框中生成一些缺失值
import random
df['value'] = [random.randint(1,100) for i in range(len(df))]
df.loc[[2,10,45,87], 'value'] = np.nan

# 1. 缺失值处理  
更多内容可参考官方文档：https://pandas.pydata.org/pandas-docs/stable/user_guide/missing_data.html

## 1.1 判断缺失值

In [4]:
# 检查数据中是否含有任何缺失值：
df.isnull().values.any()

True

In [5]:
# 查看每列数据缺失值：
df.isnull().sum(axis=0)

#行缺失值
#df.isnull().sum(axis=1)

createTime    0
education     0
salary        0
value         4
dtype: int64

In [6]:
df.shape

(135, 4)

In [7]:
# 查看每列非缺失值数：
df.notnull().sum()
df.shape[0] - df.isnull().sum()

createTime    135
education     135
salary        135
value         131
dtype: int64

**筛选出含缺失值的行列**

In [8]:
#查看各列是否存在空值
df.isnull().any(axis=0)
#查看各行是否存在空值
#df.isnull().any(axis=1)

createTime    False
education     False
salary        False
value          True
dtype: bool

In [9]:
# 返回含有缺失值的列
df.loc[:, df.isnull().any(axis=0)]

,value
0,97.0
1,18.0
2,NaN
3,35.0
4,44.0
...,...
130,71.0
131,7.0
132,3.0
133,35.0


In [10]:
# 返回含有缺失值的行
df.loc[df.isnull().any(axis=1),:]

,createTime,education,salary,value
2,2020-03-16 10:46:39,不限,20k-35k,NaN
10,2020-03-16 10:34:19,本科,15k-25k,NaN
45,2020-03-16 11:01:58,不限,25k-35k,NaN
87,2020-03-16 10:26:50,本科,10k-20k,NaN


## 1.2 缺失值填充  
https://zhuanlan.zhihu.com/p/473339132

In [11]:
# 用上下平均值填充value列：
df['value'] = df['value'].fillna(df['value'].interpolate())
df.head()
# df  (注意赋值，如果不赋值，原始序列未改变)

,createTime,education,salary,value
0,2020-03-16 11:30:18,本科,20k-35k,97.0
1,2020-03-16 10:58:48,本科,20k-40k,18.0
2,2020-03-16 10:46:39,不限,20k-35k,26.5
3,2020-03-16 10:45:44,本科,13k-20k,35.0
4,2020-03-16 10:20:41,本科,10k-20k,44.0


In [12]:
# 将value列缺失值全部替换为1.0：
df.fillna(value =1.0, inplace = True)

## 1.3 更改缺失值

In [13]:
# 将value列第10行到13行设置为缺失值：
df.loc[9:12,'value'] = np.nan

# 将第三行设置为缺失值：
df.loc[:2] = np.nan

## 1.4 删除缺失值

In [14]:
# 删除所有存在缺失值的行：
df.dropna(axis=0, how='any', inplace=True)
# 删除所有有缺失值的行
df.dropna() # -- 默认axis=0
# 删除所有有缺失值的列
df.dropna(axis='columns')
df.dropna(axis=1)
# 删除所有值缺失的行
df.dropna(how='all')
# 删除至少有两个非缺失值的行
df.dropna(thresh=2)
# 指定判断缺失值的列范围
df.dropna(subset=['education', 'value'])
# 使删除和的结果生效
df.dropna(inplace=True)
# 指定列的缺失值删除
df.value.dropna()

3       35.0
4       44.0
5       97.0
6       95.0
7       49.0
       ...  
130     71.0
131      7.0
132      3.0
133     35.0
134    100.0
Name: value, Length: 128, dtype: float64

# 2. 数据去重

In [15]:
# drop_duplicates()
df.drop_duplicates(['createTime'])

,createTime,education,salary,value
3,2020-03-16 10:45:44,本科,13k-20k,35.0
4,2020-03-16 10:20:41,本科,10k-20k,44.0
5,2020-03-16 10:33:48,本科,10k-18k,97.0
6,2020-03-16 10:11:54,硕士,16k-30k,95.0
7,2020-03-16 09:49:12,本科,10k-15k,49.0
...,...,...,...,...
128,2020-03-16 10:57:27,本科,15k-30k,76.0
129,2020-03-16 09:46:26,本科,15k-25k,15.0
130,2020-03-16 11:36:07,本科,10k-18k,71.0
131,2020-03-16 09:54:47,硕士,25k-50k,7.0


# 3.  数据合并  

我们手动创建两个数据框，和前一节的数据框一样，第一个数据集包括被试的姓名，年龄，和快乐得分；  

第二个数据框仍是关于这一批被试的数据，包含了被试的姓名，工作时长，和成绩；  

自然地，我们可以根据被试姓名，将这两个数据框合在一起，共同分析数据。  


关于数据合并的更多用法可参考：  
https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [16]:
# 创建两个dataframe
data1 = {"subID":['tom', 'jerry', 'elsa', 'white', 'kim', 'greg', 'alice', 'yeri', 'mary', 'david'],
       "age":[26, 20, 26, 24, 22, 25, 28, 20, 30, 24], 
       "happiness":[4, 2, 6, 2, 1, 2, 2, 3, 3, 6]}
data2 = {"subID":['tom', 'jerry', 'elsa', 'white', 'kim', 'greg', 'alice', 'yeri', 'mary', 'david'],
       "workhour":[8, 3, 10, 12, 12, 1, 3, 5, 7, 9],
       "grade":[90, 80, 70, 90, 84, 85, 75, 88, 98, 76]}

df_handy1 = pd.DataFrame(data1)
df_handy2 = pd.DataFrame(data2)

## 3.1 DataFrame.merge()  
- df = pd.merge(left, right, how = "inner",on = "None")  
- 参数：  
    - left: 左表。也就是第一个df。  
    - right：右表。也就是第二个df。  
    - how: 和concat里面的"join"类似，表示“如何合并两表。  
        - 1）left: 只使用左表的键。  
        - 2）right：只使用右表的键。  
        - 3）inner: 使用左右表键的交集。  
        - 4）outer：使用左右表键的并集。  
    - on: 表示按照哪一个键来进行合并。  
- 类似于关系型数据库的连接方式，可以根据一个或多个键将不同的DatFrame连接起来。该函数的典型应用场景是，针对同一个主键存在两张不同字段的表，根据主键整合到一张表里面。

In [17]:
# 按照单列合并数据：
df = pd.merge(df_handy1, df_handy2, on='subID')
df

,subID,age,happiness,workhour,grade
0,tom,26,4,8,90
1,jerry,20,2,3,80
2,elsa,26,6,10,70
3,white,24,2,12,90
4,kim,22,1,12,84
5,greg,25,2,1,85
6,alice,28,2,3,75
7,yeri,20,3,5,88
8,mary,30,3,7,98
9,david,24,6,9,76


# 4. 描述统计  

接下来我们可以对刚刚生成的新数据集进行描述统计  

描述统计的其他教程可参考：  
https://pandas.pydata.org/docs/getting_started/intro_tutorials/06_calculate_statistics.html

## 4.1 对单列进行描述统计  

比如我们关注的是成绩这一列：

In [18]:
print('grade列均值:',df['grade'].mean()) # 均值
print('全体平均数:',df.mean().mean()) # 全体平均数
print('grade列中位数:',df['grade'].median()) # 中位数
print('grade列方差:',df['grade'].var()) # 方差
print('grade列标准差:',df['grade'].std()) # 标准差
print('grade列最大值:',df['grade'].max()) # 最大值
print('grade列最小值:',df['grade'].min()) # 最小值

grade列均值: 83.6
全体平均数: 29.549999999999997
grade列中位数: 84.5
grade列方差: 71.15555555555557
grade列标准差: 8.435375246872871
grade列最大值: 98
grade列最小值: 70


/tmp/ipykernel_196/352484928.py:2: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  print('全体平均数:',df.mean().mean()) # 全体平均数


## 4.2 同时对多列进行描述统计  

比如我们想同时对成绩和工作时间进行描述统计，可以使用以下几种方法

In [19]:
# 方法一：
df[['grade', 'workhour']].agg([np.sum, np.mean, np.median, np.min, np.max, np.std, np.var])

,grade,workhour
sum,836.000000,70.000000
mean,83.600000,7.000000
median,84.500000,7.500000
min,70.000000,1.000000
max,98.000000,12.000000
std,8.435375,3.887301
var,71.155556,15.111111


In [20]:
# 方法二：
df[['grade', 'workhour']].agg(['sum', 'mean', 'median', 'min', 'max', 'std', 'var'])

,grade,workhour
sum,836.000000,70.000000
mean,83.600000,7.000000
median,84.500000,7.500000
min,70.000000,1.000000
max,98.000000,12.000000
std,8.435375,3.887301
var,71.155556,15.111111


## 4.3 对grade列求平均，对workhour列求和

In [21]:
# 方法一：分开再汇总
grade_mean = df['grade'].mean()
workhour_sum = df['workhour'].sum()
grade_mean, workhour_sum

(83.6, 70)

In [22]:
# 方法二：agg()函数
df.agg({'grade' : np.mean, 'workhour' : np.sum})

grade       83.6
workhour    70.0
dtype: float64

## 4.4 使用 .describe()

In [23]:
df[['grade','workhour']].describe()

,grade,workhour
count,10.000000,10.000000
mean,83.600000,7.000000
std,8.435375,3.887301
min,70.000000,1.000000
25%,77.000000,3.500000
50%,84.500000,7.500000
75%,89.500000,9.750000
max,98.000000,12.000000


# 5. 分组计算  

- 在处理时，我们通常需要根据某条件进行分组计算，比如计算不同处理条件下的因变量平均值。在这里我们使用了一个新的数据集“titanic”，与泰坦尼克号事件有关。  


- 参考教程：https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html

In [24]:
titanic = pd.read_csv("/home/mw/input/titanic/train.csv")
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## 5.1 分组后运算

In [25]:
# 以性别分组
titanic.groupby('Sex')

In [26]:
# 查看分组情况
titanic.groupby('Sex').groups

{'female': [1, 2, 3, 8, 9, 10, 11, 14, 15, 18, 19, 22, 24, 25, 28, 31, 32, 38, 39, 40, 41, 43, 44, 47, 49, 52, 53, 56, 58, 61, 66, 68, 71, 79, 82, 84, 85, 88, 98, 100, 106, 109, 111, 113, 114, 119, 123, 128, 132, 133, 136, 140, 141, 142, 147, 151, 156, 161, 166, 167, 172, 177, 180, 184, 186, 190, 192, 194, 195, 198, 199, 205, 208, 211, 215, 216, 218, 229, 230, 233, 235, 237, 240, 241, 246, 247, 251, 254, 255, 256, 257, 258, 259, 264, 268, 269, 272, 274, 275, 276, ...], 'male': [0, 4, 5, 6, 7, 12, 13, 16, 17, 20, 21, 23, 26, 27, 29, 30, 33, 34, 35, 36, 37, 42, 45, 46, 48, 50, 51, 54, 55, 57, 59, 60, 62, 63, 64, 65, 67, 69, 70, 72, 73, 74, 75, 76, 77, 78, 80, 81, 83, 86, 87, 89, 90, 91, 92, 93, 94, 95, 96, 97, 99, 101, 102, 103, 104, 105, 107, 108, 110, 112, 115, 116, 117, 118, 120, 121, 122, 124, 125, 126, 127, 129, 130, 131, 134, 135, 137, 138, 139, 143, 144, 145, 146, 148, 149, 150, 152, 153, 154, 155, ...]}

In [27]:
# 查看分组内容
titanic.groupby('Sex').get_group("female")

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C
...,...,...,...,...,...,...,...,...,...,...,...,...
880,881,1,2,"Shelley, Mrs. William (Imanita Parrish Hall)",female,25.0,0,1,230433,26.0000,NaN,S
882,883,0,3,"Dahlberg, Miss. Gerda Ulrika",female,22.0,0,0,7552,10.5167,NaN,S
885,886,0,3,"Rice, Mrs. William (Margaret Norton)",female,39.0,0,5,382652,29.1250,NaN,Q
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S


In [28]:
# 根据性别分组后计算年龄平均值
titanic.groupby("Sex")["Age"].mean()

Sex
female    27.915709
male      30.726645
Name: Age, dtype: float64

## 5.2 reset_index  
当使用 groupby 后，DataFrame 中的列用于分组的列会成为索引的一部分。通过 reset_index()，这些索引会被还原为普通的列，并且会重新生成默认的整数索引。

In [29]:
titanic.groupby("Sex")["Age"].mean().reset_index()

,Sex,Age
0,female,27.915709
1,male,30.726645


对比输出结果a 与 b  
关于更多对索引的理解可参照：https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.reset_index.html

In [30]:
a = titanic.groupby("Sex")["Age"].mean()
print(a)

Sex
female    27.915709
male      30.726645
Name: Age, dtype: float64


In [31]:
b= titanic.groupby("Sex")["Age"].mean().reset_index()
print(b)

      Sex        Age
0  female  27.915709
1    male  30.726645
